In [95]:
import pickle
import numpy as np
from sklearn.metrics import fbeta_score
from tqdm import tqdm

In [2]:
with open('../output/raw_pred_model-0.14135.pkl', 'rb') as f:
    ypred_train, ypred_valid, ytrain, yvalid = pickle.load(f)

In [6]:
print(ypred_train.shape, ypred_valid.shape, ytrain.shape, yvalid.shape)

(16000, 17) (4000, 17) (16000, 17) (4000, 17)


In [55]:
y_pred = np.concatenate((ypred_train, ypred_valid))
y_true = np.concatenate((ytrain, yvalid))

In [63]:
y_pred_i = (y_pred > .3).astype(int)
score = fbeta_score(y_true, y_pred_i, beta=2, average='samples')
print(score)

0.880477794585


In [57]:
thresholds = np.random.rand(17)

In [58]:
thresholds

array([ 0.2603882 ,  0.0449709 ,  0.10830222,  0.90053088,  0.14372665,
        0.57078818,  0.70073661,  0.61841861,  0.04085771,  0.41047524,
        0.02012859,  0.83583299,  0.14369802,  0.7603525 ,  0.67420828,
        0.94262221,  0.14753108])

In [77]:
def fbeta_with_thresholds(thresh, y_true=None, y_pred=None):
    if isinstance(thresh, float):
        thresh = np.ones(17)*thresh
    
    y_pred_t = y_pred.copy()
    for i in range(y_pred.shape[1]):
        y_pred_t[:,i] = y_pred[:,i] > thresh[i]
        #print(y_pred)
    
    y_pred_i = y_pred_t.astype(int)
    score = fbeta_score(y_true, y_pred_i, beta=2, average='samples')
    #print(score)
    return score*-1

In [78]:
fbeta_with_thresholds(0.3, y_true, y_pred)

-0.88047779458491238

In [79]:
fbeta_with_thresholds(thresholds, y_true, y_pred)

-0.7971339431216925

In [80]:
from scipy.optimize import minimize

In [85]:
minimize(fbeta_with_thresholds, x0=thresholds, args=(y_true, y_pred), tol=1e-9)

      fun: -0.7971339431216925
 hess_inv: array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

In [114]:
opt_thresh = list(np.ones(17)*.2)
opt_score = fbeta_with_thresholds(opt_thresh, y_true, y_pred)
for i in range(len(opt_thresh)):
    print(i)
    for t in tqdm(np.arange(0.01, .9, .011)):
        tmp_thresh = opt_thresh.copy()
        tmp_thresh[i]  = t
        new_score = fbeta_with_thresholds(tmp_thresh, y_true, y_pred)
        if new_score < opt_score :
            opt_thresh[i] = t
            opt_score = new_score

0


100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:01<00:00, 42.12it/s]


1


100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:01<00:00, 42.30it/s]


2


...

100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:01<00:00, 43.04it/s]


15


100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:01<00:00, 42.25it/s]


16


100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:01<00:00, 42.99it/s]


In [115]:
print(opt_thresh)

[0.30699999999999994, 0.075999999999999984, 0.17499999999999996, 0.08699999999999998, 0.20000000000000001, 0.31799999999999995, 0.17499999999999996, 0.11999999999999997, 0.31799999999999995, 0.30699999999999994, 0.13099999999999998, 0.21899999999999997, 0.21899999999999997, 0.31799999999999995, 0.20799999999999996, 0.25199999999999995, 0.22999999999999995]


In [116]:
print(opt_score)

-0.887820732767
